In [1]:
import json
import sys
import glob
import os.path
import statistics
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
predict_list  = []

# x_min, x_max, y_min, y_max = 0, 7, 54, 74
def pvPlot1(pitch, time):
    plt.plot(time, pitch, 'r.')
    plt.axis([0, 220, 54, 74])
    plt.show()

def pvPlot2(sample_pitch, sample_time):
    plt.plot(sample_time, sample_pitch, 'r.')
    plt.axis([0, 7, 54, 74])
    plt.show()

def pvPlay(is_value, sample_time):
    plt.bar(sample_time, is_value, width=0.05)
    plt.show()

def notePlot(start_t: list, end_t: list, note: list):
    for (s, e, n) in zip(start_t, end_t, note):
        plt.plot([float(s), float(e)], [float(n), float(n)], color='b')
        plt.plot(s, n, 'b>')
    plt.axis([0, 7, 54, 74])
    plt.show()

def pv2note(pitch, time, threshold=1.5):
    assert len(pitch)==len(time) and threshold>0
    segement_list = []
    base_val = 0.0
    start_time = 0.0
    for val, now_time in zip(pitch, time):
        if len(segement_list) == 0 and val != 0:
            segement_list.append(val)
            base_val = val
            start_time = now_time
        elif abs(val - base_val) < threshold and val != 0:
            segement_list.append(val)
        elif val != 0:
            predict_list.append([start_time, now_time, np.median(segement_list)])
            start_time = now_time
            segement_list = [val]
            base_val = val
        else:
            if len(segement_list) > 0:
                predict_list.append([start_time, now_time, np.median(segement_list)])
                segement_list = []
    if len(segement_list) > 0:
        predict_list.append([start_time, now_time, np.median(segement_list)])

    # print(predict_list)


if __name__ == "__main__":

    allFiles = glob.glob('./MIR-ST500/*/*_vocal.json')
    id = 1

    fp = open("all_vocal_output.txt", "a+") #adds to the file
    fp.truncate(0)

    for file in allFiles:
        with open(file , 'r') as reader:
            jf = json.loads(reader.read())

        print(file)

        time = np.zeros(len(jf))
        for i in range(len(jf)):
            time[i] = jf[i][0]

        pitch = np.zeros(len(jf))
        for i in range(len(jf)):
            pitch[i] = jf[i][1]

        pv2note(pitch, time)
        

        fp.write('\n'.join(str(id)))
        id += 1
        fp.write('\n'.join(str(i) for i in predict_list))

    fp.close()